# cs361 Assignment 1
### Name: Lin Lin
### UPI: llin829

--------------------------------------------------------------------------------------------------------------------------------------------------------
### Import Dataset

In [3]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [89]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
mushroom = fetch_ucirepo(id=73) 
  
# data (as pandas dataframes) 
X = mushroom.data.features 
y = mushroom.data.targets 
X.head()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,x,s,n,t,p,f,c,n,k,e,...,s,w,w,p,w,o,p,k,s,u
1,x,s,y,t,a,f,c,b,k,e,...,s,w,w,p,w,o,p,n,n,g
2,b,s,w,t,l,f,c,b,n,e,...,s,w,w,p,w,o,p,n,n,m
3,x,y,w,t,p,f,c,n,n,e,...,s,w,w,p,w,o,p,k,s,u
4,x,s,g,f,n,f,w,b,k,t,...,s,w,w,p,w,o,e,n,a,g


In [12]:
import pandas as pd
import numpy as np
import math

### Rename dataframe

In [17]:

df = pd.read_csv("C:/Users/heave/Desktop/CS361/A1/agaricus-lepiota.csv")
column_names = [
    "class",
    "cap-shape",
    "cap-surface",
    "cap-color",
    "bruises?",
    "odor",
    "gill-attachment",
    "gill-spacing",
    "gill-size",
    "gill-color",
    "stalk-shape",
    "stalk-root",
    "stalk-surface-above-ring",
    "stalk-surface-below-ring",
    "stalk-color-above-ring",
    "stalk-color-below-ring",
    "veil-type",
    "veil-color",
    "ring-number",
    "ring-type",
    "spore-print-color",
    "population",
    "habitat"
]

df.columns = column_names
df.head()

,class,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
1,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
2,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
3,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
4,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g


### Deal with missing value

In [340]:
# Replace missing values with mojority class
X = X.apply(lambda col: col.fillna(col.mode()[0]))

# Double check the number of missing values in X and y
print(X.isna().sum())
print(y.isna().sum())

cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64
poisonous    0
dtype: int64


### Split data into training set and test set

In [325]:
# Get the total number of samples
num_samples = X.shape[0]
train_size = int(num_samples * 0.9)  # 90% of the dataset

# Split the data and convert to format of Numpy
X_train = X[:train_size].to_numpy()
y_train = y[:train_size].to_numpy()
X_test = X[train_size:].to_numpy()
y_test = y[train_size:].to_numpy()

# Print each set shape
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(7311, 22)
(7311, 1)
(813, 22)
(813, 1)


## Task 1
### Q(A)

### Calculate Entropy

In [305]:
def entropy(labels):
    
        # If the label is pure or length < 1 then return 0
        if len(labels) <= 1 or len(np.unique(labels)) == 1: 
            return 0  

        # Count the occurrences of each class in the dataset
        counts = np.unique(labels, return_counts=True)[1]  
    
        # Calculate probabilities of each class
        probabilities = counts / len(labels)  

        # Calculate Entropy
        entropy = -np.sum(probabilities * np.log2(probabilities))
    
        # Return entropy
        return  entropy  

### Calculate Information gain

In [332]:
def information_gain(X, y, feature_idx):
    
        # Original entropy of the target variable
        original_entropy = entropy(y) 

         # Unique values and their counts in the feature column eg. ['a', 'f'] [500, 100]
        unique_values, counts = np.unique(X[:, feature_idx], return_counts=True) 

        # Initialize weighted entropy
        weighted_entropy = 0 

         # Total number of samples
        total_count = len(y)  
        
        # Iterate each unique value 
        for i in range(len(unique_values)):
            value = unique_values[i]  # Current unique value
            count = counts[i]  # Count of the current unique value
            
            # Indices where the feature column matches the current unique value
            value_indices = (X[:, feature_idx] == value)
            
            # Subset of targets where the feature column matches the current unique value
            target_subset = y[value_indices]
            
            # Calculate entropy for the current subset of targets
            subset_entropy = entropy(target_subset)
            
            # Calculate the proportion of samples with the current unique value
            value_proportion = count / total_count
            
            # Add the weighted entropy of the current subset to the total weighted entropy
            weighted_entropy += value_proportion * subset_entropy

            # # Calculate information gain
            ig = original_entropy - weighted_entropy
        
        return ig

### Find out best feature to split

In [329]:
def best_split(X, y):
    # Initailze information gain to 0
    best_gain = 0 

    # Initailze best_feature to None
    best_feature = None  

    # Get number of features
    number_of_features = X_train.shape[1]

    # Find out best information gain and best feature
    for i in range(number_of_features):
        info_gain = information_gain(X, y, i)
        if info_gain > best_gain:
            best_gain = info_gain
            best_feature = i
        
    return best_feature, best_gain

### Decision Tree construct

In [390]:
# Defines a DecisionNode class, which represents a node in my decision tree
class DecisionNode:
    def __init__(self, split_feature = None, gain = None, decision = None):
        # Stores the feature used for splitting at this node. If the node is a leaf, this will be None.
        self.split_feature = split_feature

        # Stores the decision or prediction made at this node if it is a leaf node. 
        self.decision = decision

        # Stores the information gain achieved by splitting on split_feature.
        self.gain = gain

        # Initializes an empty dictionary to store child nodes. Keys are the values of split_feature, 
        # and values are DecisionNode instances representing the subsequent nodes in the tree.
        self.branches = {}


class DecisionTree: 
    def __init__(self, stopping_depth=None):
        #Initializes the root of the tree as None. It will be set when the tree is built.
        self.root = None 
        self.stopping_depth = stopping_depth # Set up maximum depth

    def fit(self, X, y):
        # Starts the recursive tree-building process from the root
        self.root = self._build_tree(X, y)

    def _build_tree(self, X, y, current_depth=0):
        # Base case for recursion, if all targets in y are the same or the current depth equals the stopping depth, return a leaf node
        if len(np.unique(y)) == 1 or current_depth == self.stopping_depth:
            return Node(label=np.unique(y)[0])

        # Assign best_feature and best_gain values from best_split function
        best_feature, best_gain = best_split(X, y)  
        node = Node(feature=best_feature, information_gain = best_gain)

        for value in np.unique(X[:, best_feature]):
            # For each unique value in the best feature, partition the dataset
            indices = np.where(X[:, best_feature] == value)[0]
            X_sub, y_sub = X[indices], y[indices]
            
            # Recursively build child nodes for each partition
            node.children[value] = self._build_tree(X_sub, y_sub, current_depth + 1)
        return node

    # Predicts the labels for each instance in X
    def predict(self, X):
        return np.array([self._predict_sample(self.root, sample) for sample in X])

    def _predict_sample(self, node, sample):
        # Recursively traverses the tree to predict the label for a single instance
        if node.label is not None:
            return node.label
        feature_value = sample[node.feature]
        if feature_value in node.children:
            # If the feature value exists in children, move to the corresponding child node
            return self._predict_sample(node.children[feature_value], sample)
        else:
            return None # If the feature value does not lead to a child, return None or a default decision

    # Prints the structure of the decision tree
    def print_tree(self, node=None, indent=""):
        if node is None:
            node = self.root  
        
        if node.label is not None:
            print(f"{indent}Leaf: label={node.label}")
        else:
            feature_name = column_names[node.feature + 1]
            print(f"{indent}Node: feature={feature_name}, gain={node.information_gain}")
            
            for value, child in node.children.items():
                print(f"{indent}  Value {value}:")
                self.print_tree(child, indent + "    ")

In [351]:
# Assuming X_train, y_train, and X_test are defined
tree = DecisionTree(stopping_depth = None)
tree.fit(X_train, y_train)
tree.print_tree()

Node: feature=odor, gain=0.9011649407217686
  Value a:
    Leaf: label=e
  Value c:
    Leaf: label=p
  Value f:
    Leaf: label=p
  Value l:
    Leaf: label=e
  Value m:
    Leaf: label=p
  Value n:
    Node: feature=spore-print-color, gain=0.16243334997752942
      Value b:
        Leaf: label=e
      Value h:
        Leaf: label=e
      Value k:
        Leaf: label=e
      Value n:
        Leaf: label=e
      Value o:
        Leaf: label=e
      Value r:
        Leaf: label=p
      Value w:
        Node: feature=habitat, gain=0.3418228442877441
          Value d:
            Node: feature=gill-size, gain=0.5032583347756457
              Value b:
                Leaf: label=e
              Value n:
                Leaf: label=p
          Value g:
            Leaf: label=e
          Value l:
            Node: feature=cap-color, gain=0.6631968402398287
              Value c:
                Leaf: label=e
              Value n:
                Leaf: label=e
              Value w:
      

**Discussion:**
The decision tree output shous that the "odor" is the most significant feature, with a high information gain, indicating its critical role in distinguishing between edible and poisonous mushrooms. Then splits on features like "spore-print-color" and "habitat" further refine the classification, it structured tree's ability to capture patterns in the dataset for accurate classification.

### Q(B)

In [334]:
tree_stop2 = DecisionTree(stopping_depth=2)
tree_stop2.fit(X_train, y_train)
tree_stop2.print_tree() # Print tree when stopping depth is 2

Node: feature=odor, gain=0.9011649407217686
  Value a:
    Leaf: label=e
  Value c:
    Leaf: label=p
  Value f:
    Leaf: label=p
  Value l:
    Leaf: label=e
  Value m:
    Leaf: label=p
  Value n:
    Node: feature=spore-print-color, gain=0.16243334997752942
      Value b:
        Leaf: label=e
      Value h:
        Leaf: label=e
      Value k:
        Leaf: label=e
      Value n:
        Leaf: label=e
      Value o:
        Leaf: label=e
      Value r:
        Leaf: label=p
      Value w:
        Leaf: label=e
      Value y:
        Leaf: label=e
  Value p:
    Leaf: label=p
  Value s:
    Leaf: label=p
  Value y:
    Leaf: label=p


In [337]:
tree_stop3 = DecisionTree(stopping_depth = 3)
tree_stop3.fit(X_train, y_train)
tree_stop3.print_tree() # Print tree when stopping depth is 3

Node: feature=odor, gain=0.9011649407217686
  Value a:
    Leaf: label=e
  Value c:
    Leaf: label=p
  Value f:
    Leaf: label=p
  Value l:
    Leaf: label=e
  Value m:
    Leaf: label=p
  Value n:
    Node: feature=spore-print-color, gain=0.16243334997752942
      Value b:
        Leaf: label=e
      Value h:
        Leaf: label=e
      Value k:
        Leaf: label=e
      Value n:
        Leaf: label=e
      Value o:
        Leaf: label=e
      Value r:
        Leaf: label=p
      Value w:
        Node: feature=habitat, gain=0.3418228442877441
          Value d:
            Leaf: label=e
          Value g:
            Leaf: label=e
          Value l:
            Leaf: label=e
          Value p:
            Leaf: label=e
          Value w:
            Leaf: label=e
      Value y:
        Leaf: label=e
  Value p:
    Leaf: label=p
  Value s:
    Leaf: label=p
  Value y:
    Leaf: label=p


In [338]:
tree_stop4 = DecisionTree(stopping_depth = 4)
tree_stop4.fit(X_train, y_train)
tree_stop4.print_tree() # Print tree when stopping depth is 4

Node: feature=odor, gain=0.9011649407217686
  Value a:
    Leaf: label=e
  Value c:
    Leaf: label=p
  Value f:
    Leaf: label=p
  Value l:
    Leaf: label=e
  Value m:
    Leaf: label=p
  Value n:
    Node: feature=spore-print-color, gain=0.16243334997752942
      Value b:
        Leaf: label=e
      Value h:
        Leaf: label=e
      Value k:
        Leaf: label=e
      Value n:
        Leaf: label=e
      Value o:
        Leaf: label=e
      Value r:
        Leaf: label=p
      Value w:
        Node: feature=habitat, gain=0.3418228442877441
          Value d:
            Node: feature=gill-size, gain=0.5032583347756457
              Value b:
                Leaf: label=e
              Value n:
                Leaf: label=p
          Value g:
            Leaf: label=e
          Value l:
            Node: feature=cap-color, gain=0.6631968402398287
              Value c:
                Leaf: label=e
              Value n:
                Leaf: label=e
              Value w:
      

**Disbussion:**

**Stopping depth 2:** The tree first splitting on "odor" feature and at "n" branch further spliting on "spore-print-color" color. It generates a simpler and more interpretable model, this level of stopping depth avoid overfitting and produce more generalized outcomes with potentially higher bias but lower variance. 

**Stopping depth 3:** The tree starts to capture more detailed patterns without becoming overly complex, potentially offering a good balance between bias and variance. This might be the best stopping depth for achieving high accuracy and maintaining good generalization.

**Stopping depth 4:** The tree becomes more detailed and might start to fit more closely to the training data. This could lead to higher accuracy on the training set but might result in overfitting, affecting performance on unseen data.

### Q(C)

In [391]:
from sklearn.metrics import accuracy_score

# Prediction on test set and calculate accuracy
predictions = tree.predict(X_test)
accuracy = accuracy_score(y_test, predictions)

predictions_stop2 = tree_stop2.predict(X_test)
accuracy_stop2 = accuracy_score(y_test, predictions_stop2)

predictions_stop3 = tree_stop3.predict(X_test)
accuracy_stop3 = accuracy_score(y_test, predictions_stop3)

predictions_stop4 = tree_stop4.predict(X_test)
accuracy_stop4 = accuracy_score(y_test, predictions_stop4)

print("Accuracy of tree:", accuracy)
print("Accuracy at stopping depth 2:", accuracy_stop2)
print("Accuracy at stopping depth 3:", accuracy_stop3)
print("Accuracy at stopping depth 4:", accuracy_stop4)

Accuracy of tree: 1.0
Accuracy at stopping depth 2: 0.992619926199262
Accuracy at stopping depth 3: 0.992619926199262
Accuracy at stopping depth 4: 1.0


**Discussion:** The score of accuracy from stopping depths 2 and 3 to 4 might be seen as an improvement. A high accuracy (about 1.0) indicates that the model performs well on the test data, correctly generalizing from the patterns learned during training process without considering too much issues of overfitting. 

### Q(D)

In [392]:
# k-fold cross validation
def evaluate_decision_tree(X, y, k, stopping_depth):
    # Initialize the fold size by dividing the total number of samples by k
    fold_size = len(X) // k

    # Create an array of indices for the dataset and shuffle it to randomize the fold 
    indices = np.arange(len(X))
    np.random.shuffle(indices)

    # Set up a list to store the accuracy of each fold
    accuracies = []
    
    for fold in range(k):
        # Determine the start and end indices of the test set for the current fold 
        start, end = fold * fold_size, (fold + 1) * fold_size

        # Extract the indices for the current test set, and the remaining indices will form the training set
        test_indices = indices[start:end]
        train_indices = np.concatenate([indices[:start], indices[end:]])

        # Using the indices, split the dataset into training and test sets for this fold
        X_train, y_train = X[train_indices], y[train_indices]
        X_test, y_test = X[test_indices], y[test_indices].flatten()  # Ensure y_test is 1D

        # Initialize the decision tree with the specified stopping depth and fit it to the training data
        tree = DecisionTree(stopping_depth=stopping_depth)
        tree.fit(X_train, y_train)

        # Use the trained tree to make predictions on the test set
        predictions = tree.predict(X_test)

        # Calculate the accuracy for this fold as the mean of correct predictions
        accuracy = np.mean(predictions == y_test)
        accuracies.append(accuracy)
        
     # Append the calculated accuracy to the list of accuracies for each fold
    mean_accuracy = np.mean(accuracies)
    return mean_accuracy

# Calculate the mean accuracy across all folds to get the overall performance
mean_accuracy = evaluate_decision_tree(X, y, k=5, stopping_depth=3)
print(f"Mean accuracy across 5 folds: {mean_accuracy}")

Mean accuracy across 5 folds: 0.9942544459644322


**Discussion:** 

The dataset was shuffled and divided into 5 equal-sized folds, ensuring that each fold serves as the test set once. 

For each fold, the DecisionTree was trained on 80% of the data and tested on the remaining 20%. This process was repeated for each fold, ensuring every data point had a chance to be in the test set exactly once. 

For each fold, accuracy was calculated as the ratio of correct predictions to the total number of predictions in the test set. This provided a measure of how well the decision tree was able to generalize from the training data to unseen data.

The accuracies across all folds were averaged to obtain a mean accuracy, providing a single performance for the model across the entire dataset. 

The mean accuracy of 0.994 across 5 folds indicates that the DecisionTree model performed well in classifying the test data. This suggests that the model has effectively captured the underlying patterns and relationships in the dataset.

## Task 2
### Q(A)

**Answer:** 

Changing the splitting criterion in a decision tree can significantly change the structure and performance of the resulting model. Specifically, the decision tree algorithm may selects different the feature and the specific value of that feature to divide the dataset at each node.
Different criteria can lead to different splits, affecting the depth, balance, and generalization ability of the tree.

Gini impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. Trees using the Gini impurity tend to grow balanced trees and are good for datasets with multiple classes. A Gini score of 0 indicates perfect purity, where all instances in the node belong to a single class, whereas a Gini score of 0.5.

In the context of the mushroom classification problem, Gini Impurity helps to identify the most informative features that best separate the edible and poisonous mushrooms at each node of the decision tree. For example, if the 'odor' feature significantly decreases the Gini Impurity compared to other features, the decision tree will prioritize splitting the dataset based on 'odor' values. Hence, if features like 'odor' can almost perfectly classify mushrooms as edible or poisonous, the resulting tree might be shallower with clear rules. 

### Q(B)

Throughout my evaluation method, both training and cross-validation accuracies are high in my decision tree model. It suggests a favorable scenario where my model has learned the underlying patterns in the data well and can generalize these patterns to unseen data. It's complex enough to capture the necessary patterns in the data without being over complex that it overfits to the noise of the training set. 